In [38]:
import igraph
import dgl
import torch
import time
import os
import networkx as nx
from datetime import datetime
import pandas as pd
import sys
sys.path.append("./FastCover/")
sys.path.append("./models/")
from utils import *
from GRAT import GRAT3
import warnings
warnings.filterwarnings('ignore')
from models import GNNModel
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [150]:
Distributions = {'GAT':None, 'GCN':None, 'GraphConv':None, 'GRAT':None}

In [151]:
use_cuda = False

PATH_TO_TEST = "./BRKGA/instances/txt/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Graphs_short = [
 'ego-facebook.txt',
 'gemsec_facebook_artist.txt',
 'graph_actors_dat.txt',
 'graph_CA-AstroPh.txt',
 'graph_CA-CondMat.txt',
 'graph_CA-GrQc.txt',
 'graph_CA-HepPh.txt',
 'graph_CA-HepTh.txt',
 'graph_dolphins.txt',
 'graph_Email-Enron.txt',
 'graph_football.txt',
 'graph_jazz.txt',
 'graph_karate.txt',
 'graph_ncstrlwg2.txt',
 'soc-gplus.txt',
 'socfb-Brandeis99.txt',
 'socfb-Mich67.txt',
 'socfb-nips-ego.txt']

Graphs_large = ['Amazon0302.txt',
 'Amazon0312.txt',
 'Amazon0505.txt',
 'Amazon0601.txt',
 'com-youtube.ungraph.txt',
 'com-dblp.ungraph.txt',
 'loc-gowalla_edges.txt',
 'deezer_HR.txt',
 'musae_git.txt']

Graphs = [Graphs_short[10]]
Graphs

['graph_football.txt']

## Models

In [152]:
PATH_SAVED_TRAINS = "models/runs/"
RUNS_LIST = [run for run in os.listdir(PATH_SAVED_TRAINS) if ".pt" in run]

SEEDS = []
MODELS = []
for run_name in RUNS_LIST:
    SEEDS.append(run_name.split("_")[2])
    MODELS.append(run_name.split("_")[0])
    
records = []
Total = len(Graphs)
    
for run_name, model, seed in zip(RUNS_LIST, MODELS, SEEDS):
    print()
    print(f"Evaluation of model: {model}")
    print()
    
    net = GNNModel(c_in = 1, c_hidden = 100, c_out = 2, num_layers = 2, layer_name = model, dp_rate=0.1)
    net.load_state_dict(torch.load(PATH_SAVED_TRAINS+run_name))
    
    if use_cuda:
        net.cuda()

    c = 1
    for file in Graphs:
            name = file.split(".")[0].replace("graph_", "")

            graph = igraph.Graph().Read_Edgelist(PATH_TO_TEST + file)
            data = Convert2DataSet([graph.to_networkx()], [[]])[0]

            G = graph.to_networkx().to_undirected()

            n = len(G.nodes())
            
            out = net(data.x, data.edge_index).max(1)[0]
            Distributions[model] = out.detach().numpy().copy()



Evaluation of model: GAT


Evaluation of model: GCN


Evaluation of model: GraphConv



## FastCover

In [153]:
PATH_SAVE_TRAINS = "./FastCover/runs/"

FEATURE_TYPE = "1"
HIDDEN_FEATS = [32]*6
input_dim = 32
use_cuda = False
directed_test = False

RUNS_LIST = [run for run in os.listdir(PATH_SAVE_TRAINS) if ".pt" in run]

SEEDS = []
MODELS = []
for run_name in RUNS_LIST:
    SEEDS.append(run_name.split("_")[2])
    MODELS.append(run_name.split("_")[0])


records = []

Total = len(Graphs)
    
for run_name, model, seed in zip(RUNS_LIST, MODELS, SEEDS):
    print()
    print(f"Evaluation of model: {model}, seed: {seed} in {run_name}")
    print()
    
    if model == 'GRAT':
        net = GRAT3(*HIDDEN_FEATS)
        net.load_state_dict(torch.load(PATH_SAVE_TRAINS+run_name))
    if use_cuda:
        net.cuda()

    c = 1
    for file in Graphs:
            name = file.split(".")[0].replace("graph_", "")

            graph = igraph.Graph().Read_Edgelist(PATH_TO_TEST + file)

            dglgraph = get_rev_dgl(graph, FEATURE_TYPE, input_dim, directed_test, use_cuda)
            
            G = graph.to_networkx().to_undirected()

            out = net.grat(dglgraph, dglgraph.ndata['feat']).squeeze(1)
            
            Distributions[model] = out.detach().numpy().copy()
            
            del dglgraph
            gc.collect()



Evaluation of model: GRAT, seed: 13 in GRAT_seed_13_10-15_17-40.pt



In [154]:
Distributions = pd.DataFrame.from_dict(Distributions)
Distributions.head()

,GAT,GCN,GraphConv,GRAT
0,21.158415,16.863871,27.619099,7.931191
1,21.101004,20.934233,70.218887,27.926386
2,19.552031,15.126616,26.230764,-13.576478
3,19.815014,19.314949,70.360405,-1.995505
4,21.601580,20.917109,66.493698,-17.297899


In [155]:
Distributions_ = Distributions**2
Distributions_ = Distributions_ /Distributions_.max()
Distributions_.head()

,GAT,GCN,GraphConv,GRAT
0,0.944606,0.301089,0.000404,0.035121
1,0.939487,0.463975,0.002610,0.435431
2,0.806619,0.242250,0.000364,0.102911
3,0.828464,0.394973,0.002621,0.002223
4,0.984591,0.463216,0.002341,0.167061


In [211]:
k = 20

In [212]:
NodeOrder = np.argsort(-np.array(nx.degree(G)).T[1])[:k]
D = Distributions_.iloc[NodeOrder,:]

NodeOrderReverse = np.argsort(np.array(nx.degree(G)).T[1])[:k]
D_reverse = Distributions_.iloc[NodeOrderReverse,:]
D_reverse.columns = D_reverse.columns + "_r"

In [214]:
fig = ff.create_distplot(np.array(D).T, D.columns, show_hist = False, histnorm = 'probability density')
fig.update_layout(title = f"{Graphs[0]} top {k} degree nodes")
fig.show()

In [163]:
fig = ff.create_distplot(np.array(D_reverse).T, D_reverse.columns, show_hist=False)
fig.update_layout(title = f"{Graphs[0]} top {k} less degree nodes")
fig.show()

### Por nodos

In [160]:
fig = go.Figure()
for k_ in Distributions_.columns:
    fig.add_trace(go.Bar(
        x = np.array(NodeOrder, dtype="str"),
        y = D[k_],
        name=k_
    ))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                 title = f"{Graphs[0]} top {k} degree nodes")
fig.show()

In [161]:
fig = go.Figure()
for k_ in Distributions_.columns:
    fig.add_trace(go.Bar(
        x = np.array(NodeOrderReverse, dtype="str"),
        y = D_reverse[k_],
        name=k_
    ))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                 title = f"{Graphs[0]} top {k} less degree nodes")
fig.show()